# Executor do Projeto GeoPandas (Alura) — com gráficos salvos

Este executor:
1. Baixa `Projeto_completo.ipynb` do repositório da Alura;
2. Insere um *preamble* para garantir `%matplotlib inline` e DPI alto;
3. Executa **todas** as células com o Papermill, gerando `Projeto_completo_executado.ipynb` com todas as saídas;
4. Extrai **todas as imagens** geradas pelos gráficos e salva em `figures/`;
5. Exporta também para **HTML** (`Projeto_completo_executado.html`).

> Dica: Rode em Google Colab (Runtime → Run all).

In [ ]:
# Parâmetros
INPUT_NB_URL = "https://raw.githubusercontent.com/alura-cursos/curso_geopandas/main/Projeto_completo.ipynb"
INPUT_NB_PATH = "Projeto_completo.ipynb"
OUTPUT_NB_PATH = "Projeto_completo_executado.ipynb"
OUTPUT_HTML_PATH = "Projeto_completo_executado.html"
FIG_DIR = "figures"
print("Notebook origem:", INPUT_NB_URL)
print("Destino (.ipynb):", OUTPUT_NB_PATH)
print("Destino (.html):", OUTPUT_HTML_PATH)
print("Pasta de figuras:", FIG_DIR)


In [ ]:
import sys, subprocess
def pip_install(*packages):
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + list(packages)
    print("Instalando:", " ".join(packages))
    subprocess.check_call(cmd)

# Dependências para execução e pós-processamento
pip_install("papermill", "requests", "nbformat", "nbconvert")
# Obs.: o notebook da Alura instala geopandas durante a execução.


In [ ]:
import requests, json
r = requests.get(INPUT_NB_URL)
r.raise_for_status()
nb_json = r.json()
with open(INPUT_NB_PATH, "w", encoding="utf-8") as f:
    json.dump(nb_json, f)
print("Baixado:", INPUT_NB_PATH)


In [ ]:
# Insere um preâmbulo na primeira célula para garantir inline backend
import nbformat as nbf
nb = nbf.read(INPUT_NB_PATH, as_version=4)
preamble = nbf.v4.new_code_cell("""
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
""")
nb.cells.insert(0, preamble)
nbf.write(nb, INPUT_NB_PATH)
print("Preâmbulo inserido e salvo em:", INPUT_NB_PATH)


In [ ]:
# Executa o notebook com Papermill
import papermill as pm
print("Executando notebook. Isso pode levar alguns minutos...")
pm.execute_notebook(
    INPUT_NB_PATH,
    OUTPUT_NB_PATH,
    kernel_name="python3",
)
print("Concluído! Gerado:", OUTPUT_NB_PATH)


In [ ]:
# Extrai todas as imagens do notebook executado para FIG_DIR
import os, base64, nbformat
from pathlib import Path

Path(FIG_DIR).mkdir(exist_ok=True)
nb = nbformat.read(OUTPUT_NB_PATH, as_version=4)
count = 0
for i, cell in enumerate(nb.cells):
    if cell.get("outputs"):
        for j, out in enumerate(cell["outputs"]):
            data = out.get("data", {}) if isinstance(out, dict) else {}
            for mime in ("image/png", "image/jpeg", "image/svg+xml"):
                if mime in data:
                    count += 1
                    ext = {"image/png": ".png", "image/jpeg": ".jpg", "image/svg+xml": ".svg"}[mime]
                    fname = f"fig_{i:03d}_{j:02d}{ext}"
                    fpath = os.path.join(FIG_DIR, fname)
                    if mime == "image/svg+xml":
                        svg_text = data[mime]
                        if isinstance(svg_text, list):
                            svg_text = "".join(svg_text)
                        with open(fpath, "w", encoding="utf-8") as fh:
                            fh.write(svg_text)
                    else:
                        img_b64 = data[mime]
                        if isinstance(img_b64, list):
                            img_b64 = "".join(img_b64)
                        with open(fpath, "wb") as fh:
                            fh.write(base64.b64decode(img_b64))
print(f"Imagens extraídas: {count}")
print("Diretório:", os.path.abspath(FIG_DIR))


In [ ]:
# Exporta também para HTML para visualização rápida
from nbconvert import HTMLExporter
import nbformat

nb = nbformat.read(OUTPUT_NB_PATH, as_version=4)
html_exporter = HTMLExporter()
body, _ = html_exporter.from_notebook_node(nb)
with open(OUTPUT_HTML_PATH, "w", encoding="utf-8") as f:
    f.write(body)
print("Exportado HTML:", OUTPUT_HTML_PATH)


In [ ]:
# Cria um ZIP com o .ipynb executado e a pasta de figuras
import zipfile, os
zip_name = "Projeto_completo_executado_bundle.zip"
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as z:
    z.write(OUTPUT_NB_PATH)
    if os.path.isdir(FIG_DIR):
        for root, _, files in os.walk(FIG_DIR):
            for f in files:
                p = os.path.join(root, f)
                z.write(p)
print("Gerado bundle:", zip_name)


In [ ]:
# (Opcional) Download automático no Google Colab
try:
    from google.colab import files
    files.download(OUTPUT_NB_PATH)
    files.download(OUTPUT_HTML_PATH)
    files.download("Projeto_completo_executado_bundle.zip")
except Exception:
    import pathlib
    print("Se não estiver no Colab, baixe manualmente os arquivos gerados na pasta atual:")
    for p in [OUTPUT_NB_PATH, OUTPUT_HTML_PATH, "Projeto_completo_executado_bundle.zip"]:
        print(" -", pathlib.Path(p).resolve())
